Pororo Summarization Abstract

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141822 sha256=f82c883552f32f5f5844113228af637dd9346405a1b77a3abb7a27a061b0c580
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [4]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize

import re
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [5]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [6]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [7]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [8]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [9]:
train.head()

,uid,title,region,context,summary,total
1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


In [10]:
test.head()

,uid,title,region,context,total
2000,2000,제235회 본회의 제1차(2012.06.21.),음성,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...,제235회 본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001,2001,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군..."
2002,2002,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록..."
2003,2003,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결..."
2004,2004,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분..."


In [11]:
testcopy = test['total']

In [12]:
testcopy

2000    제235회    본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군...
2002    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록...
2003    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결...
2004    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분...
                              ...                        
2501    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제1항, 제17...
2502    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제2항, 농협 ...
2503    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제3항, 음성군...
2504    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제4항, 제4기...
2505    제174회    본회의 제2차(2006.10.16.) 음성 의사일정 제5항, 휴회의...
Name: total, Length: 506, dtype: object

In [13]:
!pip install --upgrade pororo

In [14]:
!pip install kollocate

In [15]:
!pip install koparadigm

In [16]:
!pip install g2pk

In [17]:
testcopy_total_list = testcopy.tolist()

In [18]:
testcopy_total_list[0]

'제235회    본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 제1차 정례회 제1차 본회의를 개의하겠습니다. 먼저 의회사무과장으로부터 보고가 있겠습니다. 의회사무과장입니다. 먼저 지난 제234회 임시회 제1차 본회의에서 의결하여 이송한 안건에 대해서 보고를 드리겠습니다. 2012년 5월 29일 제1차 본회의에서 의결하여 이송한 음성군 지방공무원 정원 조례 일부개정조례안은 2012년 6월 15일자로 공포되었습니다. 다음은 제235회 제1차 정례회 제1차 본회의와 관련하여 보고 드리겠습니다. 「지방자치법」제44조 및 「음성군 회기와 그 운영 등에 관한 조례」제4조제1항에 따라 정례회를 개최하고자 6월 10일자로 집회공고를 하였습니다. 6월 14일 이대웅 의원님 외 2분의 의원님으로 환경분야 현지확인 특별위원회 구성 결의안이, 손달섭 의원님 외 2분의 의원님으로부터 제234회 임시회 시 활동한 주요사업 현지확인 결과 보고의 건이 각각 접수되었습니다. 음성군수로부터 2012년 6월 14일자로 2011회계 세입ㆍ세출 결산 승인안이, 6월 19일자로 2011년도 예비비 지출 승인안이 각각 접수되어 오늘 제1차 정례회를 개최하게 되었습니다. 이상으로 보고를 마치겠습니다. 사무과장님 수고 많이 하셨습니다.'

In [19]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.6.0
cuda version: 10.2
cudnn version:7605


In [20]:
# 235
from pororo import Pororo

summ_abs = Pororo(task="summarization", model="abstractive", lang="ko")
abs_result = []
for i in range(0,236):
  abs_result.append(summ_abs(
      testcopy_total_list[i],
      beam=5,
      len_penalty=0.7,
      no_repeat_ngram_size=3,
      top_k=100,
      top_p=0.7))

[Korean Sentence Splitter]: Initializing Kss...
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [21]:
# abs_result

In [22]:
# 236
abs_result.append(summ_abs(
    testcopy_total_list[235],
    beam=5,
    len_penalty=0.7,
    no_repeat_ngram_size=3,
    top_k=100,
    top_p=0.7))

In [23]:
# 237
for i in range(237,len(testcopy_total_list)):
  abs_result.append(summ_abs(
      testcopy_total_list[i],
      beam=5,
      len_penalty=0.7,
      no_repeat_ngram_size=3,
      top_k=100,
      top_p=0.7))

In [24]:
abs_result[:10]

['군 의회는 제235회 임시회 제1차 본회의에서 의결한 안건에 대해서 보고 드리겠습니다.',
 '군의회 제235회 제1차 정례회 회기결정건에서 환경분야 현지확인 특위 구성의 건, 주요사업 현지확인 결과 보고의 건과 조례안 2건, 2011년도 예비비 지출 승인안, 2012년도 하반기 군정주요업무 계획 보고 등이 상정되었다.',
 '본회의 의사일정 제2항, 회의록 서명의원 선출의 건에서  조천희 의원님이  손달섭 의원님을  선출하고자 하는데,  「없습니다」하는 의원 여러분   이의가 없으므로  조 의원님, 손달석 의원님이 회의록서명의원으로 선출되었음을 선포한다.',
 '의사일정 제3항, 예산결산특별위원회 구성의 건을 상정하였으며  제가 제의한 예산결간특별위원회 위원을 지명하도록 하겠다.',
 '제235회 제1차 정례회 제1 1차 본회의를 맞이하여 음성군 환경분야 현지확인을 발의한 이대웅 의원은 「없습니다」 이의가 없으므로 환경분야 특별위원회 구성결의안은 가결되었음을 선포했다.',
 '달섭 의원은 「2012 주요사업 현지확인 특별위원회」에 활동하면서  현지확인 결과 보고서 1페이지 활동개요는 유인물을 참고하며 군민들의 불편사항을 해결하고 사업비 확보가 최우선 과제임을 밝히며 사업 추진 시 계획 수립을 철저히 해 예산이 중복투자되거나 낭비되는 사례가 없도록 특단의 대책을 강구해 주시기 바란다고 말했다.',
 '22회 음성군의회 임시회 제1차 본회의에서 의회사무과장 최태옥씨는 집행부로 이송한 안건에 대하여 보고드리며, 의회사무원과장 서태삼씨는 의안 상정현황을 보고드리겠습니다.',
 '제322회 음성군의회 임시회 회기결정 건을 상정하며 4월 22일부터 27일까지 6일간 의사일정을 결정하기로 하였으며 이의가 없으므로 가결되었음을 선포한다.',
 '본회의 의사일정 제2항, 회의록 서명의원 선출의 건에서  김영섭 부의장님, 김영호 의원님이 이의 없으십니까? (「없습니다」하는 의원 있음) 이의가 없으므로 회의록서명의원으로 선출되었음을 선포한다.',
 '본회의도 제1회 세입ᆞ세

In [25]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [26]:
submission

,uid,summary
0,id_2000-AGENDA_1,요약문
1,id_2000-AGENDA_2,요약문
2,id_2000-AGENDA_3,요약문
3,id_2000-AGENDA_4,요약문
4,id_2000-AGENDA_5,요약문
...,...,...
501,id_2096-AGENDA_2,요약문
502,id_2096-AGENDA_3,요약문
503,id_2096-AGENDA_4,요약문
504,id_2096-AGENDA_5,요약문


In [27]:
abs_result = pd.Series(abs_result)

In [28]:
submission['summary'] = abs_result

In [29]:
len(submission)

506

In [30]:
submission

,uid,summary
0,id_2000-AGENDA_1,군 의회는 제235회 임시회 제1차 본회의에서 의결한 안건에 대해서 보고 드리겠습니다.
1,id_2000-AGENDA_2,"군의회 제235회 제1차 정례회 회기결정건에서 환경분야 현지확인 특위 구성의 건, ..."
2,id_2000-AGENDA_3,"본회의 의사일정 제2항, 회의록 서명의원 선출의 건에서 조천희 의원님이 손달섭 ..."
3,id_2000-AGENDA_4,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정하였으며 제가 제의한 예산결간..."
4,id_2000-AGENDA_5,제235회 제1차 정례회 제1 1차 본회의를 맞이하여 음성군 환경분야 현지확인을 발...
...,...,...
501,id_2096-AGENDA_2,당초74회 음성군의회 임시회 회기연장은 농협 축산물공판장 이전관련 현지확인 특별위원...
502,id_2096-AGENDA_3,본태완 위원장님께서는 「농협 축산물공판장 이전관련 현지확인 특별위원회 활동기간 연장...
503,id_2096-AGENDA_4,문화공보과장은 문화공료과에서 제출한 대소도서관 관리운영조례안에 대하여 설명 드리도록...
504,id_2096-AGENDA_5,보건기구에 의하면 제4기 음성군 지역보건 의료계획은 지역보건법 제3조 및 동법시행령...


In [31]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/dacon_pororo_abstract.csv', index=False)